# Network and relations making

In [1]:
import subprocess
import os
import time
from yml_create_functions import make_yml, make_network_agents_yml

In [2]:
name_simulation = 'simulation_tutorial'

## Network Agent Parameters

The schema can be changed in schema/schema.py. Here, you can change the parameters of the default schema

In [3]:
network_agents_parameters = {
    "default": {
        "id_message": "NaN",
        "has_tv": "false",
        "cause": -1,
        "method": "NaN",

        "type": "dumb",
        "response": "NaN",
        "stance": "agree",
        "repost": "NaN",
        "parent_id": "NaN"

    },

    "DumbViewer": [
        {"weight": 2, "type": "dumb"},
        {"weight": 2, "type": "dumb", "has_tv": "true"}
    ],
    "HerdViewer": [
        {"weight": 2, "type": "herd", "stance": "against"},
        {"weight": 2, "type": "herd", "has_tv": "true"}
    ],
    "WiseViewer": [
        {"weight": 1, "type": "wise", "stance": "against"},
        {"weight": 1, "type": "wise", "has_tv": "true", "stance": "against"}
    ],

}

## Other Parameters

In [4]:
parameters = {
    "default_state": "{}",
    "load_module": "schema", # path from yml to schema
    "environment_agents": "[]",
    "environment_class": "schema.NewsEnvironmentAgent", # path from yml to environment class
    "environment_params": {
        "prob_neighbor_spread": 0.02,  # 006
        "prob_tv_spread": 0.006,  # 001
        "prob_neighbor_cure": 0.006,  # 001
        "prob_backsliding": 0.01,  # 003
        "prob_dead": 0.006,  # 001,
        "prob_repost": 0.8,
        "mean_time_connection": 50,
        "var_time_connection": 30
    },
    "interval": 1,
    "max_time": 100,
    "name": name_simulation,
    "network_params": {
        "generator": "barabasi_albert_graph",
        "n": 100,
        "m": 5
    },
    "num_trials": 1
}

# Prob responses parameters

In [5]:
# Define your probabilities here
prob_response = {"dumb": {"support": 0.4, "deny": 0.3, "question": 0, "comment": 0.2},
                "herd": {"support": 0.25, "deny": 0.25, "question": 0.25, "comment": 0.25},
                "wise": {"support": 0.2, "deny": 0.2, "question": 0.3, "comment": 0.3}
                }
types = list(prob_response.keys())
responses = list( prob_response[types[0]].keys() )
for type_i in types:
    for response in responses:
        name_i = f"prob_{type_i}_{response}"
        prob_i = prob_response[type_i][response]
        # Here, we set each probability
        parameters["environment_params"][name_i] = prob_i

## Simulation execution

In [6]:
parameters_i = parameters.copy()
network_agents_parameters_i = network_agents_parameters.copy()
data = make_yml(parameters_i)
data += make_network_agents_yml(network_agents_parameters_i)

yml_path = os.path.join('schema', f'{name_simulation}.yml') # YML path
with open(yml_path, 'w') as file:
    file.write(data)

command = "soil"
start = time.time()

output = subprocess.check_output([command, yml_path])
end = time.time()
seconds_simulation = str(end-start)
print("SIMULATION'S SECONDS:", seconds_simulation)

SIMULATION'S SECONDS: 7.230015754699707


# Conversation Making

In [7]:
from get_data.get_data import get_pivoted_data
from post.templates import POST_TEMPLATE, REPLY_TEMPLATE, INSTRUCTIONS_TEMPLATE
from spelling_checker.spelling_checker import correctness_prompt, correctness_percentage
from anytree import Node, RenderTree, LevelOrderIter
from post.post import Post
from IA.ia import send_prompt

## Output sqlite path

In [8]:
analysis_path = os.path.join('soil_output', name_simulation)
sql_table_path = f'{name_simulation}_trial_0.sqlite'

## Get Data

In [9]:
attributes = ['cause', 'method', 'response', 'stance', 'repost']

data = get_pivoted_data(analysis_path, sql_table_path, attributes)

## Define News

In [10]:
NEWS = 'Chilean Government Announces Ambitious Plan for Sustainable Energy Transition'
SHOW_TREE = True

## Create the Network's Tree

In [11]:
root = Post(0, message=NEWS, step=0, post_template=POST_TEMPLATE,
             reply_template=REPLY_TEMPLATE,
             instructions_template=INSTRUCTIONS_TEMPLATE, news=NEWS)

In [12]:
list_nodes = [root]
for i in range(len(data)):
    aux = data[i]
    owner = aux[0]
    step = int(aux[1])
    id_message = aux[2]
    parent = list_nodes[int(aux[3])]
    state = aux[4] #unused
    attr_dict = {}
    for index, attr in enumerate(attributes):
        index +=5
        attr_dict[attr] = aux[index] 
    
    ### Specific for repost attribute
    if attr_dict["repost"] == 'true':
        attr_dict["repost"] = True
    elif attr_dict["repost"] == 'false':
        attr_dict["repost"] = False
    else:
        print(data[i])
        print(attr_dict["repost"])
        raise ValueError("Error repost format")
    ### 

    message = ''

    list_nodes.append(Post(name=id_message, parent=parent, owner=owner, step=step, message=message, 
                             post_template=POST_TEMPLATE, reply_template=REPLY_TEMPLATE,
                             instructions_template=INSTRUCTIONS_TEMPLATE, news=NEWS,
                             **attr_dict
                             )
    )

## Associate a Written Post to each Message

In [13]:
from parameters import API_KEY
import openai
openai.api_key = API_KEY

In [14]:
TEMPERATURE = 1.5
list_messages = []
for node_i in LevelOrderIter(root):
    if node_i == root:
        continue
    elif node_i.repost:
        continue

    instructions, prompt = node_i.get_prompt(language='english', 
                min_caract=130, max_caract=250,
                user_description='average toxic and angry social media user')
    answer = send_prompt(instructions, prompt, temp=TEMPERATURE, max_tokens=1000)
    correctness = correctness_percentage(answer)
    while correctness >= 0.2:
        print("Correction")
        correction_prompt = correctness_prompt(NEWS, answer)
        print(correction_prompt)
        answer = send_prompt(instructions, correction_prompt, temp=TEMPERATURE, max_tokens=1000)
        correctness = correctness_percentage(answer)
    
    node_i.set_message(answer)
    list_messages.append((node_i, prompt))

Correction

The news is as follows: 'Chilean Government Announces Ambitious Plan for Sustainable Energy Transition'.
Please rewrite the related news post below: 'Wow, just heard about it... Chilean Gov pulling another sunny surprise - Plans for a Switch-up to Sustainable Energy! BRUH, they've proclaimed an AMBITIOUS makeover and revolutionizing the energy scene! Now they're looking forward ✨🌍 Bright future on the horizon for my guys in Chile!🔥🌞 #SustainableEnergy #EnergyMakeover #BrightFuture 🙌💯

It seems like they really strive to prioritize renewable energy sources.But ugh, I wonder how this plan will affect the GDP 😰, they probably going for higher electricity tariffs too🤔💸 Aren't we paying enough already? SO UNDOUBTEDLY Higher Government bills coming straight at our 🏠 wallets to strive 100% renewable 🚀😤 Meanwhile, general public awaits 💭 like all seemed...Sebastian Piñera to work wonders? We oughta go Solar instead⁉ Dare I brighten up hope for our flaming sky💔 Just make electricity

## Plot the Network in a Tree

In [15]:
if SHOW_TREE:
    for pre, _, node in RenderTree(root):
        if node.name == 0:
            print(f"{pre}NEWS: {message}")
            continue

        if node.repost:
            message = "repost"
        else:
            message = "'"+node.message+"'"
        print(f"{pre}{node.owner} (t={node.step}) {message}")

NEWS: 
├── 69 (t=30) 'Oh wow, the Chilean government just dropped a bomb! 🤯 They're stepping it up with an ambitious plan for a sustainable energy transition 🔋💡 It's about time we start using clean energy and leaving old destructive techniques in the dust 💨 Chile is on a roll, leading Latin America in solar power ☀️☀️ Talk about chopping those carbon emissions down! 🌿🍃 #SustainableFuture #GoChile 🇨🇱!'
├── 18 (t=43) 'Dang, everyone needs to take note! The Chilean gov is going hard on green energy💚 They just proclaimed this 🔥 plan for a sustainable energy shift. 🌬️ With a focus on solar and wind power, 🌞 it's gonna bring huge environment benefits and support the economy. 🌍 Hats off, Chile! 🇨🇱 #FutureIsGreen'
│   ├── 94 (t=46) repost
│   ├── 65 (t=68) repost
│   ├── 30 (t=79) 'Um, sorry, but I smell some Greenwashing ain't right. 😡 Chile's shiny new renewable energy plan may sound all perfect to you, but what about the cost? 🤷‍♀️ Why don't they show us the facts? How much will it hit taxp